In [1]:
import json
import math
from collections import OrderedDict
import torch
from torch import nn, Tensor
from typing import Union, Tuple, List, Iterable, Dict
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.optim import AdamW
from torch.utils.data import DataLoader
from scipy.stats import pearsonr, spearmanr
import numpy as np
import gzip, csv
import pandas as pd
from tqdm.auto import tqdm

torch.manual_seed(0)
np.random.seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
! pip3 install datasets

In [ ]:
# %pip install transformers
# from transformers import AutoTokenizer
# # If you can not find all the bugs, use the line below for AutoModel
# #from transformers import AutoModel


In [ ]:
! pip3 install avalanche-lib

In [ ]:
! pip3 install torch transformers

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from collections import OrderedDict
import xformers.ops as xops


def gelu(x):
    """Implementation of the gelu activation function."""
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class Config(object):
    """Configuration class to store the configuration of a `BertModel`."""
    def __init__(self,
                 vocab_size,
                 hidden_size=512,
                 num_hidden_layers=6,
                 num_attention_heads=8,
                 intermediate_size=2048,
                 dropout_prob=0.1,
                 max_position_embeddings=512,
                 type_vocab_size=2,
                 initializer_range=0.02):
        """Constructs Config for BertModel."""
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.intermediate_size = intermediate_size
        self.hidden_dropout_prob = dropout_prob
        self.attention_probs_dropout_prob = dropout_prob
        self.max_position_embeddings = max_position_embeddings
        self.type_vocab_size = type_vocab_size
        self.initializer_range = initializer_range

    @classmethod
    def from_dict(cls, dict_object):
        """Constructs Config from a Python dictionary of parameters."""
        config = Config(vocab_size=None)
        for (key, value) in dict_object.items():
            config.__dict__[key] = value
        return config

class LayerNorm(nn.Module):
    """Layer normalization module."""
    def __init__(self, hidden_size, variance_epsilon=1e-12):
        """Constructs LayerNorm object for Transformer layer in BERT model."""
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(hidden_size))
        self.beta = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = variance_epsilon

    def forward(self, x):
        """Forward pass of the LayerNorm layer."""
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta

class MLP(nn.Module):
    """Feed forward network with gelu activation."""
    def __init__(self, hidden_size, intermediate_size):
        """Constructs MLP object for Transformer layer in BERT model."""
        super(MLP, self).__init__()
        self.dense_expansion = nn.Linear(hidden_size, intermediate_size)
        self.dense_contraction = nn.Linear(intermediate_size, hidden_size)

    def forward(self, x):
        """Forward pass of the MLP layer."""
        x = self.dense_expansion(x)
        x = self.dense_contraction(gelu(x))
        return x

class Layer(nn.Module):
    """The Transformer layer."""
    def __init__(self, config):
        """Constructs Layer object for Transformer layer in BERT model based on config."""
        super(Layer, self).__init__()

        self.hidden_size = config.hidden_size
        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

        self.attn_out = nn.Linear(config.hidden_size, config.hidden_size)
        self.ln1 = LayerNorm(config.hidden_size)

        self.mlp = MLP(config.hidden_size, config.intermediate_size)
        self.ln2 = LayerNorm(config.hidden_size)

    def split_heads(self, tensor, num_heads, attention_head_size):
        """Split hidden_size into num_heads * attention_head_size and transpose into shape [batch, num_heads, seq_len, attention_head_size]."""
        new_shape = tensor.size()[:-1] + (num_heads, attention_head_size)
        tensor = tensor.view(*new_shape)
        return tensor.permute(0, 2, 1, 3).contiguous()

    def merge_heads(self, tensor, num_heads, attention_head_size):
        """Transpose and then reshape into shape [batch, seq_len, hidden_size]."""
        tensor = tensor.permute(0, 2, 1, 3).contiguous()
        new_shape = tensor.size()[:-2] + (num_heads * attention_head_size,)
        return tensor.view(new_shape)

    # def attn(self, q, k, v, attention_mask):
    def attn(self, q, k, v):
        """Attention mechanism for the Transformer layer."""
        #mask = attention_mask == 1
        #mask = mask.unsqueeze(1).unsqueeze(2)

        # s = torch.matmul(q, k.transpose(-1, -2))
        # s = s / math.sqrt(self.attention_head_size)

        # s = torch.where(s, torch.tensor(float('-inf')))

        # p = F.softmax(s, dim=-1)
        # p = self.dropout(p)

        # a = torch.matmul(p, v)
        a = xops.memory_efficient_attention(q, k, v, p=0.1)
        return a

    #def forward(self, x, attention_mask):
    def forward(self, x):
        """Forward pass of the Transformer layer in BERT."""
        q, k, v = self.query(x), self.key(x), self.value(x)

        q = self.split_heads(q, self.num_attention_heads, self.attention_head_size)
        k = self.split_heads(k, self.num_attention_heads, self.attention_head_size)
        v = self.split_heads(v, self.num_attention_heads, self.attention_head_size)

        #a = self.attn(q, k, v, attention_mask)
        a = self.attn(q, k, v)
        a = self.merge_heads(a, self.num_attention_heads, self.attention_head_size)
        a = self.attn_out(a)
        a = self.dropout(a)
        a = self.ln1(a + x)

        m = self.mlp(a)
        m = self.dropout(m)
        m = self.ln2(m + a)

        return m

class Bert(nn.Module):
    def __init__(self, config_dict):
        super(Bert, self).__init__()
        self.config = Config.from_dict(config_dict)  # Create an instance of Config
        postion = nn.Embedding(self.config.max_position_embeddings, self.config.hidden_size)
        postion.weight.data = self.random_sinusoidal_positional_embedding(self.config.max_position_embeddings, self.config.hidden_size)
        self.embeddings = nn.ModuleDict({
            'token': nn.Embedding(self.config.vocab_size, self.config.hidden_size, padding_idx=0),
            'position': postion,
            #'position': nn.Embedding(self.config.max_position_embeddings, self.config.hidden_size),
            'token_type': nn.Embedding(self.config.type_vocab_size, self.config.hidden_size),
        })

        self.ln = LayerNorm(self.config.hidden_size)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        #update batch, include cross attention for first layer(just for the first layer)
        self.layers = nn.ModuleList([
            Layer(self.config) for _ in range(self.config.num_hidden_layers)
        ])

        self.pooler = nn.Sequential(OrderedDict([
            ('dense', nn.Linear(self.config.hidden_size, self.config.hidden_size)),
            ('activation', nn.Tanh()),
        ]))

        # Add a classifier layer for classification
        self.classifier = nn.Linear(self.config.hidden_size, 20)

    def random_sinusoidal_positional_embedding(self, token_sequence_size, token_embedding_dim, n=10000.0):

        if token_embedding_dim % 2 != 0:
            raise ValueError("Sinusoidal positional embedding cannot apply to odd token embedding dim (got dim={:d})".format(token_embedding_dim))

        T = token_sequence_size
        d = token_embedding_dim #d_model=head_num*d_k, not d_q, d_k, d_v

        positions = torch.arange(0, T).unsqueeze_(1)
        embeddings = torch.zeros(T, d)

        denominators = torch.pow(n, 2*torch.arange(0, d//2)/d) # 10000^(2i/d_model), i is the index of embedding
        embeddings[:, 0::2] = torch.sin(positions/denominators) # sin(pos/10000^(2i/d_model))
        embeddings[:, 1::2] = torch.cos(positions/denominators) # cos(pos/10000^(2i/d_model))

        return embeddings

    #def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
    def forward(self, input_ids, token_type_ids=None, labels=None):    
        #input_ids = input_ids.to(device)
        position_ids = torch.arange(input_ids.size(1), dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        # if attention_mask is None:
        #     attention_mask = torch.ones_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        # position_ids = position_ids.to(device)
        # attention_mask = attention_mask.to(device)
        # token_type_ids = token_type_ids.to(device)
        # labels = labels.to(device)
        x = (self.embeddings['token'](input_ids) +
             self.embeddings['position'](position_ids) +
             self.embeddings['token_type'](token_type_ids))
        x = self.dropout(self.ln(x))

        for layer in self.layers:
            x = layer(x)

        o = self.pooler(x[:, 0])

        if labels is not None:
          # Use the classifier layer for classification
          logits = self.classifier(o)
          #loss = nn.CrossEntropyLoss()(logits, labels)
          return logits

        return x, o

    def save_model(self, path):
        """Save model to a file.

        Args:
            path (str): Path to the file where the model will be saved.
        """
        torch.save(self.state_dict(), path)

    @classmethod
    def load_model(cls, config_dict, path):
        """Load model from a file.

        Args:
            config_dict (dict): Dictionary containing the configuration of the model.
            path (str): Path to the model checkpoint.

        Returns:
            Bert: Bert model loaded from the checkpoint.
        """
        model = cls(config_dict)
        model.load_state_dict(torch.load(path))
        return model


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.2.1+cu121 with CUDA 1201 (you have 2.1.2+cu118)
    Python  3.10.11 (you have 3.10.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "c:\Users\saira\anaconda3\envs\CSCLDL\lib\site-packages\xformers\__init__.py", line 55, in _is_triton_available
    from xformers.triton.softmax import softmax as triton_softmax  # noqa
  File "c:\Users\saira\anaconda3\envs\CSCLDL\lib\site-packages\xformers\triton\softmax.py", line 11, in <module>
    import triton
ModuleNotFoundError: No module named 'triton'


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.is_available()

True

In [4]:
device

device(type='cuda')

In [ ]:
model = Bert(config_dict)
model.to(device)

In [3]:
from transformers import AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch
from transformers import BertTokenizer

# Define your custom BERT configuration
# config_dict = {
#     'vocab_size': 310000, 
#     'hidden_size': 512,
#     'num_attention_heads': 2,
#     'num_hidden_layers': 4,
#     'intermediate_size': 512,
#     'dropout_prob': 0.1,
#     'max_position_embeddings': 512,
#     'type_vocab_size': 2,
#     'initializer_range': 0.02
# }
# config_dict = {
#     'vocab_size': 30522,  # Standard for BERT-base
#     'hidden_size': 768,   # Standard for BERT-base
#     'num_attention_heads': 12,  # Standard for BERT-base
#     'num_hidden_layers': 12,     # Standard for BERT-base
#     'intermediate_size': 3072,   # Standard for BERT-base
#     'dropout_prob': 0.1,         # Commonly used dropout probability
#     'max_position_embeddings': 512,  # Standard for BERT models
#     'type_vocab_size': 2,        # Standard for BERT models
#     'initializer_range': 0.02    # Standard for BERT models
# }
config_dict = {
    'vocab_size': 30522,  # Standard for BERT-base
    'hidden_size': 384,   # Reduced hidden size
    'num_attention_heads': 4,  # Reduced attention heads
    'num_hidden_layers': 6,     # Reduced number of layers
    'intermediate_size': 1536,  # Increased intermediate size
    'dropout_prob': 0.1,         # Commonly used dropout probability
    'max_position_embeddings': 512,  # Standard for BERT models
    'type_vocab_size': 2,        # Standard for BERT models
    'initializer_range': 0.02    # Standard for BERT models
}

# Create an instance of your custom BERT model
model = Bert(config_dict)
model.to(device)
# Tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # Replace with your desired tokenizer

# Define a DataLoader for the dataset
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.tokenizer(item['text'], truncation=True, padding='max_length', return_tensors='pt', max_length=512)
        encoding['label'] = torch.tensor(item['label'])
        return encoding

# Use DynamicPaddingCollate for DataLoader
class DynamicPaddingCollate:
    def __call__(self, batch):
        return {
            'input_ids': torch.stack([sample['input_ids'].squeeze(0) for sample in batch]),
            'attention_mask': torch.stack([sample['attention_mask'].squeeze(0) for sample in batch]),
            'token_type_ids': torch.stack([sample['token_type_ids'].squeeze(0) for sample in batch]),
            'labels': torch.tensor([sample['label'] for sample in batch])
        }

# Load the dataset
dataset = load_dataset("setfit/20_newsgroups", split="train")

# Create DataLoader with DynamicPaddingCollate
train_dataset = MyDataset(dataset, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=DynamicPaddingCollate())

# Training parameters
epochs = 10
learning_rate = 3e-5

# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
criterion = nn.CrossEntropyLoss()
# Training loop


for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    total_loss = 0.0

    for batch_idx, batch in enumerate(train_dataloader, 1):
        optimizer.zero_grad()
        input_ids =batch['input_ids']
        input_ids = input_ids.to(device)
        # attention_mask = batch['attention_mask']
        # attention_mask = attention_mask.to(device)
        labels = batch['labels']
        labels = labels.to(device)
        token_type_id = batch['token_type_ids']
        token_type_id = token_type_id.to(device)
        #labels = labels.to(device)
        #
        #break
        #outputs = model(input_ids, attention_mask, token_type_id, labels)
        outputs = model(input_ids, token_type_id, labels)
        # print(outputs)
        # check = outputs
        # break
        #loss =
        loss = criterion(outputs, labels)  # Assuming your forward method returns the loss directly
        loss.backward()  
        optimizer.step()

        total_loss += loss.item()

        if batch_idx % 10 == 0:  # Print every 10 batches
            avg_loss = total_loss / batch_idx
            print(f"Batch {batch_idx}/{len(train_dataloader)} - Avg Loss: {avg_loss:.4f}")

    avg_epoch_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Avg Loss: {avg_epoch_loss:.4f}")
    curent_state = {
            'epoch': epoch + 1,
            'model_state': model.state_dict(),
            'optimizer_state': optimizer.state_dict(),
        }
    save_path = f"D:/Studies/DL/vsccode/models/my_custom_bert_modelv5.pth"
    torch.save(curent_state, save_path)
    print(f"Saved model state to'{save_path}'")
    scheduler.step()





c:\Users\saira\anaconda3\envs\CSCLDL\lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
c:\Users\saira\anaconda3\envs\CSCLDL\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
Batch 10/354 - Avg Loss: 3.0204
Batch 20/354 - Avg Loss: 3.0266
Batch 30/354 - Avg Loss: 3.0210
Batch 40/354 - Avg Loss: 3.0236
Batch 50/354 - Avg Loss: 3.0188
Batch 60/354 - Avg Loss: 3.0182
Batch 70/354 - Avg Loss: 3.0168
Batch 80/354 - Avg Loss: 3.0143
Batch 90/354 - Avg Loss: 3.0131
Batch 100/354 - Avg Loss: 3.0131
Batch 110/354 - Avg Loss: 3.0128
Batch 120/354 - Avg Loss: 3.0092
Batch 130/354 - Avg Loss: 3.0099
Batch 140/354 - Avg Loss: 3.0090
Batch 150/354 - Avg Loss: 3.0067
Batch 160/354 - Avg Loss: 3.0054
Batch 170/354 - Avg Loss: 3.0049
Batch 180/354 - Avg Loss: 3.0053
Batch 190/354 - Avg Loss: 3.0047
Batch 200/354 - Avg Loss: 3.0029
Batch 210/354 - Avg Loss: 3.0021
Batch 220/354 - Avg Loss: 3.0003
Batch 230/354 - Avg Loss: 2.9991
Batch 240/354 - Avg Loss: 2.9968
Batch 250/354 - Avg Loss: 2.9959
Batch 260/354 - Avg Loss: 2.9948
Batch 270/354 - Avg Loss: 2.9934
Batch 280/354 - Avg Loss: 2.9920


KeyboardInterrupt: 

In [8]:
model.save_model("D:/Studies/DL/vsccode/models/my_custom_bert_modelv50.pth")

In [9]:
for parameter in model.parameters():
    print(parameter)

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1580,  0.1073,  0.6626,  ..., -0.5771,  1.1372,  0.6051],
        [-0.6771, -0.0862, -0.5989,  ...,  0.5772,  1.7381,  0.6154],
        ...,
        [ 0.7708, -0.7637,  0.2595,  ..., -0.5272, -2.1669,  0.9764],
        [ 0.8387, -1.1839,  2.6811,  ...,  0.1538,  1.4220,  1.3415],
        [ 0.1027,  0.9284, -2.1318,  ...,  0.5642, -0.4453,  0.6948]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[-5.7197e-06,  1.0002e+00, -1.0539e-04,  ...,  9.9999e-01,
          2.1249e-04,  1.0004e+00],
        [ 8.4178e-01,  5.3967e-01,  8.2157e-01,  ...,  9.9980e-01,
          9.2461e-04,  1.0008e+00],
        [ 9.0880e-01, -4.1743e-01,  9.3506e-01,  ...,  1.0007e+00,
          1.5743e-04,  9.9970e-01],
        ...,
        [ 6.1545e-02,  9.9836e-01,  7.9820e-01,  ...,  9.9813e-01,
          5.2977e-02,  9.9872e-01],
        [ 8.7316e-01,  4.8733e-01,  9.4980e-01,  ...,

In [10]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
print(get_n_params(model))

26674708


In [ ]:
model.save_model("D:/Studies/DL/vsccode/models/my_custom_bert_modelv50.pth")

In [18]:
train_dataset.__getitem__(10)

{'input_ids': tensor([[  101,  1045,  2031,  1037,  2240,  2006,  1037, 26363, 10450,  7706,
         13512,  2015,  3301,  2944,  2007,  2459,  2243,  2006,  1996,  5119,
          1012,  3216,  2200,  2092,  1010,  6773,  2003,  1996,  4421,  1013,
          2829,  1013,  4589,  8105,  2041,  1010, 29324,  1037,  2978,  1997,
          3514,  1998, 16949,  2041,  1997,  3083,  2007,  2524, 16222,  2884,
          1012,  1996,  4497,  2097,  8081,  9099,  1998,  3514, 17271,  1012,
          2027,  2853,  1996,  7997,  2000,  1996,  1015,  1998,  2069,  3954,
          1012,  2027,  2215,  1002,  4090,  2683,  2629,  1010,  1998,  1045,
          2572,  3241,  2062,  2066,  1002,  1017,  2243,  1012,  2151, 10740,
          2041,  2045,  1029,  3531, 10373,  2033,  1012,  4283,  1012,  2009,
          2052,  2022,  1037,  3835,  6540,  6775,  2000,  1996, 10506,  5017,
          1012,  2059,  1045,  1005,  2222,  2131,  1037, 14855,  2361,  7997,
          1998,  2655,  2870,  8123,  

In [27]:
# Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# print(input)
# print(target)
# output = loss(input, target)
# print("one",output)
# output.backward()
# print("one",output)
#Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# print(target)
# output = loss(input, target)
# output.backward()
# print("two",output)

tensor([[-0.2599,  1.7095, -1.8641,  0.4603,  1.0727],
        [ 1.3016,  0.2321, -1.3055, -1.0979,  0.3024],
        [-0.4994,  0.4463, -2.0962,  1.7518,  1.2974]], requires_grad=True)
tensor([2, 4, 0])
one tensor(2.9490, grad_fn=<NllLossBackward0>)
one tensor(2.9490, grad_fn=<NllLossBackward0>)


In [23]:
#outputs

tensor([[0.0563, 0.0574, 0.0524, 0.0411, 0.0941, 0.0515, 0.0538, 0.0506, 0.0443,
         0.0489, 0.0426, 0.0487, 0.0561, 0.0322, 0.0470, 0.0469, 0.0438, 0.0522,
         0.0449, 0.0352],
        [0.0521, 0.0482, 0.0492, 0.0369, 0.0977, 0.0582, 0.0564, 0.0400, 0.0496,
         0.0411, 0.0519, 0.0455, 0.0738, 0.0366, 0.0482, 0.0340, 0.0500, 0.0406,
         0.0575, 0.0323],
        [0.0590, 0.0494, 0.0491, 0.0429, 0.0783, 0.0548, 0.0565, 0.0452, 0.0483,
         0.0544, 0.0437, 0.0416, 0.0769, 0.0388, 0.0457, 0.0445, 0.0484, 0.0401,
         0.0505, 0.0319],
        [0.0510, 0.0491, 0.0431, 0.0398, 0.1017, 0.0448, 0.0531, 0.0348, 0.0533,
         0.0454, 0.0487, 0.0429, 0.0787, 0.0329, 0.0516, 0.0436, 0.0493, 0.0456,
         0.0612, 0.0293],
        [0.0507, 0.0465, 0.0489, 0.0376, 0.0945, 0.0496, 0.0660, 0.0351, 0.0511,
         0.0464, 0.0428, 0.0453, 0.0753, 0.0330, 0.0497, 0.0458, 0.0557, 0.0445,
         0.0510, 0.0305],
        [0.0600, 0.0488, 0.0499, 0.0383, 0.0946, 0.0540, 0.0

In [22]:
# magic = torch.argmax(outputs, dim = 1)
# magic

tensor([ 4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        12,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4])

In [ ]:

# Save the model
model.save_model("D:/Studies/DL/vsccode/models/my_custom_bert_modelv50.pth")

## TESTING CHECK TOMORROW

In [11]:
# # Load the model for predictions
# loaded_model = Bert.load_model(config_dict, "D:/Studies/DL/vsccode/models/my_custom_bert_modelv1.pth")

# # Perform predictions
# sentence = "I like computers"
# input_ids = tokenizer(sentence, return_tensors='pt')['input_ids']
# output = loaded_model(input_ids)
# print(torch.argmax(F.softmax(output[0], dim=1), dim=1))

tensor([[3, 0, 3, 0, 2, 3, 2, 4, 0, 3, 3, 4, 2, 3, 2, 3, 4, 3, 2, 2, 2, 3, 3, 3,
         0, 3, 2, 3, 2, 4, 3, 3, 3, 4, 3, 1, 3, 4, 4, 3, 4, 4, 3, 4, 2, 3, 2, 3,
         2, 3, 3, 4, 3, 3, 3, 1, 4, 4, 4, 0, 1, 1, 2, 3, 2, 4, 3, 0, 3, 3, 3, 0,
         2, 1, 3, 0, 1, 0, 4, 3, 1, 2, 0, 2, 4, 0, 0, 1, 3, 2, 4, 4, 3, 3, 3, 2,
         3, 3, 3, 1, 4, 3, 0, 2, 3, 3, 3, 0, 2, 4, 2, 2, 2, 3, 3, 3, 4, 0, 2, 3,
         3, 4, 4, 2, 2, 3, 4, 3, 3, 0, 0, 2, 2, 3, 1, 4, 1, 3, 4, 3, 3, 0, 3, 4,
         2, 0, 2, 4, 4, 1, 1, 0, 3, 4, 3, 4, 4, 3, 0, 3, 3, 2, 4, 3, 4, 3, 3, 3,
         3, 4, 3, 2, 2, 3, 2, 3, 0, 3, 3, 4, 4, 3, 3, 3, 2, 3, 4, 0, 2, 4, 2, 4,
         3, 4, 2, 3, 0, 2, 4, 1, 1, 4, 0, 0, 4, 3, 2, 4, 4, 4, 0, 3, 3, 4, 3, 3,
         2, 3, 4, 1, 2, 3, 3, 3, 3, 0, 3, 4, 2, 4, 4, 2, 2, 4, 3, 2, 1, 2, 4, 3,
         2, 3, 3, 2, 3, 1, 0, 4, 4, 4, 3, 2, 1, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 3,
         1, 3, 0, 0, 1, 4, 3, 3, 3, 3, 0, 3, 3, 2, 3, 3, 2, 3, 0, 3, 3, 2, 2, 2,
         4, 3, 3, 3, 3, 3, 2

In [10]:
output[0]

tensor([[[ 0.0070, -0.1704, -0.0670,  ...,  0.1719,  0.6466,  0.5834],
         [-0.4745, -0.3692, -0.0863,  ...,  0.1384,  0.7785,  0.5543],
         [-0.5119, -0.3893, -0.0829,  ...,  0.1456,  0.8330,  0.5947],
         [-0.4829, -0.3874, -0.0650,  ...,  0.1734,  0.8240,  0.5547],
         [-0.4764, -0.3769, -0.1003,  ...,  0.1372,  0.7869,  0.0832]]],
       grad_fn=<AddBackward0>)

In [14]:
check[0]

tensor([0.0868, 0.0667, 0.0448, 0.0425, 0.0855, 0.0469, 0.0439, 0.0399, 0.0453,
        0.0427, 0.0820, 0.0357, 0.0340, 0.0547, 0.0405, 0.0406, 0.0423, 0.0527,
        0.0329, 0.0395], grad_fn=<SelectBackward0>)

In [ ]:
prediction = torch.argmax(check[0])
prediction

tensor(1)

In [ ]:
check[0]

tensor([-0.3700,  0.4549,  0.0396,  0.1715,  0.2064,  0.1312,  0.3936, -0.0831,
         0.2345, -0.2406, -0.2414,  0.3022, -0.1611, -0.3026, -0.0525,  0.0938,
         0.2632, -0.0264,  0.3938,  0.2777], grad_fn=<SelectBackward0>)

In [ ]:
# torch.save(model.state_dict(), 'model_weights.pth')
# model = MyModel()  # Make sure this is the same model architecture
# model.load_state_dict(torch.load('model_weights.pth'))
# model.eval()

tensor([17,  4,  9, 13, 19, 16,  5,  1,  6,  6, 12,  3,  4, 19, 10,  4,  1, 14,
         5, 13, 19, 17,  0, 19, 16, 12,  0, 10,  6,  3,  7,  0])

In [ ]:
# Access the 19th row in the "text" column
text_19th_row = dataset['train']['text'][18]

print(text_19th_row)

In [19]:
for batch_idx, batch in enumerate(train_dataloader, 1):
  print(**batch)
  break

TypeError: 'input_ids' is an invalid keyword argument for print()

In [5]:
def position_encoding_init(n_position, d_pos_vec):
    ''' Init the sinusoid position encoding table '''

    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2*i/d_pos_vec) for i in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    return torch.from_numpy(position_enc).type(torch.FloatTensor)

In [6]:
def random_sinusoidal_positional_embedding(token_sequence_size, token_embedding_dim, n=10000.0):

    if token_embedding_dim % 2 != 0:
        raise ValueError("Sinusoidal positional embedding cannot apply to odd token embedding dim (got dim={:d})".format(token_embedding_dim))

    T = token_sequence_size
    d = token_embedding_dim #d_model=head_num*d_k, not d_q, d_k, d_v

    positions = torch.arange(0, T).unsqueeze_(1)
    embeddings = torch.zeros(T, d)

    denominators = torch.pow(n, 2*torch.arange(0, d//2)/d) # 10000^(2i/d_model), i is the index of embedding
    embeddings[:, 0::2] = torch.sin(positions/denominators) # sin(pos/10000^(2i/d_model))
    embeddings[:, 1::2] = torch.cos(positions/denominators) # cos(pos/10000^(2i/d_model))

    return embeddings

In [7]:
print(position_encoding_init(512, 512))

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 8.4147e-01,  5.6969e-01,  8.0196e-01,  ...,  1.0000e+00,
          1.0746e-08,  1.0000e+00],
        [ 9.0930e-01, -3.5090e-01,  9.5814e-01,  ...,  1.0000e+00,
          2.1492e-08,  1.0000e+00],
        ...,
        [ 6.1950e-02,  6.0239e-01,  6.5887e-01,  ...,  1.0000e+00,
          5.4698e-06,  1.0000e+00],
        [ 8.7333e-01, -3.1283e-01, -2.0968e-01,  ...,  1.0000e+00,
          5.4805e-06,  1.0000e+00],
        [ 8.8177e-01, -9.5883e-01, -9.0939e-01,  ...,  1.0000e+00,
          5.4912e-06,  1.0000e+00]])


In [8]:
print(random_sinusoidal_positional_embedding(512,512))

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  8.2186e-01,  ...,  1.0000e+00,
          1.0366e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.3641e-01,  ...,  1.0000e+00,
          2.0733e-04,  1.0000e+00],
        ...,
        [ 6.1950e-02,  9.9808e-01,  7.9822e-01,  ...,  9.9850e-01,
          5.2740e-02,  9.9861e-01],
        [ 8.7333e-01,  4.8714e-01,  9.4980e-01,  ...,  9.9850e-01,
          5.2844e-02,  9.9860e-01],
        [ 8.8177e-01, -4.7168e-01,  2.8398e-01,  ...,  9.9849e-01,
          5.2947e-02,  9.9860e-01]])
